<a id='TOC'></a>

# Cleaning Data (Part 1)
The purpose of this notebook is to read in raw excel data for multiple years, rename and trim columns, append cleaned files into a single dataframe and export this dataframe as an excel file.

# Table of Contents
1. [Setting up Python](#SettingUp)
    
    1. [Setting the Location](#SettingLoc)
    
    2. [Importing Necessary Packages](#ImportingPackages)
    
    3. [Functions](#functions)
    
    4. [Preparing for a Save](#PreparingSave)  

2. [Handling Columns](#HandlingColumns)
    
    1. [Find Unique Column Names](#FindUniqueCol)
    
    2. [Eliminate Unnecessary Columns](#DropCol)
    
    3. [Combine Synonyms](#CombineCol)

3. [Reading and Appending Data](#ReadingAppendingData)

4. [Exporting Data](#ExportingData)

<a id='SettingUp'></a>

# Setting up Python
[Top](#TOC)

[Setting the Location](#SettingLoc)
    
[Importing Necessary Packages](#ImportingPackages)
    
[Getting Data](#GettingData)
    
[Preparing for a Save](#PreparingSave)

<a id='ImportingPackages'></a>

## Importing Necessary Packages

[Top](#TOC)

[Setting Up Python](#SettingUp)

Here we import necessary packages. 
This chunk may take a while.

In [1]:
import pandas as pd
# import numpy as np
import glob,os

# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_colwidth = 50
pd.set_option('mode.sim_interactive', True)

<a id='functions'></a>

## Functions
[Back to: Top](#TOC)

[Back to: Setting Up Python](#SettingUp)

1. [xlcolshape](#xlcolshape)

2. [xluniquecol2](#xluniquecol2)

3. [colmatchtodict](#colmatchtodict)

4. [findsyn](#findsyn)

5. [readnsplit](#readnsplit)

6. [mapndrop](#mapndrop)

7. [namefile](#namefile)

<a id='xlcolshape'></a>

[Back to: Functions](#Functions)

In [2]:
def xlcolshape(file, verbose = True):
    """xlcolshape takes a file name as a string and returns the shape of the excel file"""
    assert isinstance(verbose,bool),"'verbose' must be bool not,{}".format(type(verbose))
    dictionary = {}
    for sheet in pd.ExcelFile(file).sheet_names:
        try:
            tmp = pd.read_excel(file,sheet_name =sheet).shape
            dictentry = file+'_'+sheet
            dictionary[dictentry] = tmp
            if verbose == True:
                print("Doing stuff you asked me to do for file \'{}\',sheet \'{}\' programmer person."\
                      .format(file, sheet))            
        except:
            print("This didn't work for file {}, sheet {}".format(file, sheet))
            
    return dictionary

<a id='xluniquecol2'></a>

[Back to: Functions](#Functions)

In [3]:
def xluniquecol2(file, header = 0, verbose=True):
    tmp = []
    for sheet in pd.ExcelFile(file).sheet_names:
        if (('species' in pd.read_excel(file,sheet_name=sheet, header = header).columns)\
            or('Species' in pd.read_excel(file,sheet_name=sheet, header = header).columns)):
            try:
                tmp = list(set(tmp+list(pd.read_excel(file,sheet_name=sheet).columns)))
                if verbose==True:
                    print("Doing stuff you asked me to do for file \'{}\',sheet \'{}\' programmer person."\
                          .format(file,sheet))
                res = tmp
            except:
                print("This didn't work for file {}, sheet {}".format(file,sheet))
        else:
            print("Check columns for file {}.".format(file))
            res = None
    return res
            

<a id='colmatchtodict'></a>

[Back to: Functions](#Functions)

In [4]:
def colmatchtodict(x,series, dictsource, key= None):
    """This takes a string, x, and a looks for values in a series that match that contain that string.
    Those values which match are returned as values in a python dict for the key, key.""" 
    assert isinstance(series,pd.Series)
    if key is None:
        key = x
    tmp = series[series.astype(str).str.contains(x,case = False)].tolist()
    dictsource[key] = tmp
    return dictsource
    

<a id='findsyn'></a>

[Back to: Functions](#Functions)

In [5]:
def findsyn (name,dictionary, verbose = True):
    """
    *findsyn* checks searches the values of the dict *dictionary* for the string, *name* and returns 
    the key for the key,value pair to which *name* belongs.
    """
    tmp = pd.DataFrame({'preferredcol':list(dictionary.keys()),'synonymns':list(dictionary.values())})
    try:
        res = list(tmp.preferredcol[tmp.synonymns.apply(lambda x:name in x)])[0]
    except:
        res = None
        if verbose == True:
            print("No value matching \"{}\" was found in the dictionary.".format(name))
    return res


<a id='readnsplit'></a>

[Back to: Functions](#Functions)

In [6]:
def readnsplit(file,newsourcefolder,dtype=None,verbose=True):
    """
    This function reads an excel file, splits its sheets into separate files and saves them to folder
    *newsourcefolder*.
    """
    suffix = '.'+file.split('.')[1]
    prefix = file[:-len(suffix)]
    for sheet in pd.ExcelFile(file).sheet_names:
        try:
            splitfile = newsourcefolder+'/'+prefix+'_'+sheet+suffix
            tmp = pd.read_excel(file,dtype=dtype, sheet_name=sheet).to_excel(splitfile,index=False)
            if verbose==True:
                print("Succes!  \'{}\',sheet \'{}\' has been saved to {} and the corresponding\
                google drive file as {}.".format(file,sheet,newsourcefolder,splitfile))
            continue
        except:
            print("Unable to save \'{}\',sheet \'{}\' as a separate file.".format(file,sheet))         


<a id='mapndrop'></a>

[Back to: Functions](#Functions)

In [7]:
def mapndrop(df,dictionary,verbose=True):
    """
    This function renames columns in *df* deemed synonymous according to a dict,
    *dictionary*, and drops unnecessary columns before returning the cleaner dataframe.
    """
    try:
        df.columns = pd.Series(df.columns).map(lambda x:dictionary[x])
        tmp = df
        if verbose==True:
            print("Successfully mapped columns for df.")
        dropidx =[None==col for col in list(tmp.columns)]
        tmp=tmp.drop(columns=df.columns[dropidx])
        if verbose==True:
            print("Successfully dropped unnecessary columns for df.")
    except:
        tmp = None
        print("Skipping mapndrop call for df.")
    return tmp


<a id='namefile'></a>

[Back to: Functions](#Functions)

In [8]:
def namefile(name, tzadjust=5,tzdirection = '-', adjprecision='minutes', filetype = 'csv'):
    """takes a filename and filetype, and adds a timestamp adjusted relative to gmt to a precision 
    and returns a string that concatenates them."""
    assert isinstance(name,str),"'name' must be of type str."
    assert isinstance(tzadjust,int),"'tzadjust' must be of type int"
    assert adjprecision in ['date','hour','minutes','seconds', 'max'], "'adjprecision' must be either \
    'date', hour','minutes','seconds', or 'max'"
    precision= {'max':None,'seconds':-7,'minutes':-9, 'hours':-14,'date':-20}
    if tzdirection== '-':
        timestamp = (pd.to_datetime('now')-pd.Timedelta(hours=tzadjust))
    else:
        timestamp = (pd.to_datetime('now')+pd.Timedelta(hours=int(tzadjust[1:])))
    timestamp = str(timestamp).replace(':','hrs',1).replace(':','min',1)
    timestamp = timestamp[:precision[adjprecision]]
    filename = name+'_' + timestamp+ '.' +filetype
    return filename


<a id='PreparingSave'></a>

## Preparing for a Save
[Top](#TOC)

[Setting up Python](#SettingUp)

<a id='SettingLoc'></a>

## Setting the Location
[Top](#TOC)

[Setting Up Python](#SettingUp)

These chunks identify the locations from which we can get data and to which we can save data.

### Source Data
Raw data can be found in the following locations:

In [9]:
# sourceDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
sourceDataBig = 'S:/Chris/TailDemography/TailDemography/Raw Data'
# sourceBlack = 'C:/Users/test/Desktop'

### Intermediate Source Data
Intermediate files can be found in the following locations:

In [10]:
# sourceInterDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/Intermediate Files/Source'
sourceinterDataBig = 'S:/Chris/TailDemography/TailDemography/Intermediate Files/Source'
# sourceBlack = 'C:/Users/test/Desktop'

Now we change the working directory to the source path.

In [11]:
os.chdir(sourceDataBig)

### Output Data
The cleaned data will be saved to one of these locations:

In [12]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
outputBig = 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data'
# outputBlack = 'C:/Users/test/Desktop'

### Review files

In [13]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/Files for review/Source files'
reviewfolderBig = 'S:/Chris/TailDemography/TailDemography/Files for review/Source files'

<a id='HandlingColumns'></a>

# Handling Columns
[Top](#TOC)

We don't have to look in the multiple-sheet file.  It's clear that we'll have to identify a common set of columns prior to combining these files.  Let's define a few functions to help us do this.

We will want to do the following:
1. [Find Unique Column Names](#FindUniqueCol)
2. [Eliminate Unnecessary Columns](#DropCol)
3. [Combine Synonyms](#CombineCol)

Here we use search the source path to locate and eventually read the raw data into our notebook.

In [14]:
rawfiles = glob.glob('*.xls*')
rawfiles

['CC 2000-03-modified from CC-SJ 00-03 final.xls',
 'CC 2004.xlsx',
 'CC 2015 - captures.xls',
 'CC 2016 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2005x.xls',
 'xCC2006x.xls',
 'xCC2007x.xls',
 'xCC2008x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2012x.xls',
 'xCC2013x.xls',
 'xCC2014x.xlsx']

We'll separate these into files with single or multiple sheets.

In [15]:
rawfiles_ms = [rawfiles[0],rawfiles[7]]
rawfiles_ss = list(set(rawfiles)- set(rawfiles_ms))

The names of files with multiple sheets are now in the variable *rawfiles_ms*.

In [16]:
rawfiles_ms

['CC 2000-03-modified from CC-SJ 00-03 final.xls', 'xCC2007x.xls']

The names of files with a single sheet are now in the variable *rawfiles_ss*.

In [17]:
rawfiles_ss

['xCC2005x.xls',
 'xCC2006x.xls',
 'CC 2015 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'CC 2004.xlsx',
 'xCC2011x.xls',
 'CC 2016 - captures.xls',
 'xCC2012x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2008x.xls',
 'xCC2014x.xlsx',
 'xCC2013x.xls']

Now let's take a look at the number of columns in each file. We'll start with the single sheet files, since this is the easiest.  We will use the function, *xlcolshape* to make this easier. 
When we call this function on the first of the single-sheet files, we can see that it returns a tuple in the format ('number of rows', 'number of columns'). The code for *xlcolshape* can be found in [Functions](#functions).

In [18]:
xlcolshape(rawfiles_ss[0])

Doing stuff you asked me to do for file 'xCC2005x.xls',sheet '2005' programmer person.


{'xCC2005x.xls_2005': (202, 16)}

We will apply this function to the list of files for our inspection.

In [19]:
pd.Series(rawfiles).apply(lambda x: xlcolshape(x,verbose=False))

0     {'CC 2000-03-modified from CC-SJ 00-03 final.x...
1                     {'CC 2004.xlsx_2004 ': (479, 16)}
2            {'CC 2015 - captures.xls_2015': (241, 19)}
3            {'CC 2016 - captures.xls_2016': (103, 21)}
4     {'CC 2017 Lizards - 3viii17 captures and obs.x...
5                      {'xCC2005x.xls_2005': (202, 16)}
6                      {'xCC2006x.xls_2006': (163, 16)}
7     {'xCC2007x.xls_Sheet1': (507, 16), 'xCC2007x.x...
8                      {'xCC2008x.xls_2008': (134, 20)}
9                      {'xCC2009x.xls_2009': (162, 16)}
10                   {'xCC2010x.xlsx_Sheet1': (99, 41)}
11                    {'xCC2011x.xls_Sheet1': (64, 19)}
12                      {'xCC2012x.xls_data': (85, 19)}
13             {'xCC2013x.xls_CC 2013 data': (106, 19)}
14                     {'xCC2014x.xlsx_2014': (97, 19)}
dtype: object

<a id='FindUniqueCol'></a>

## Finding Unique Columns
[Top](#TOC)

[Handling Columns](#HandlingColumns)

We'll use the function, *xluniqucol2* to extract column names and convert them to an approved set.  We'll use that function to allow us to only add unique names to a list of column names. 

Here is an example of how xluniquecol2 works for a file with one sheet.  You can find the code for *xluniquecol2* in [Functions](#functions).

In [20]:
xluniquecol2(rawfiles_ss[0],verbose=False)

['TL',
 'species',
 'VIAL',
 'paint mark',
 'TIME',
 'location',
 'NEW/recap',
 'date',
 'mass',
 'misc',
 'meters',
 'SVL',
 'painted or not',
 'TOES',
 'RTL',
 'sex']

Here is an example of how xluniquecol2 works for a file with multiple sheets.

In [21]:
xluniquecol2(rawfiles_ms[0],verbose=False)

Check columns for file CC 2000-03-modified from CC-SJ 00-03 final.xls.


['TL',
 'Unnamed: 16',
 'species',
 'VIAL',
 'paint mark',
 'TIME',
 'location',
 'NEW/recap',
 'misc',
 'date',
 'meters',
 'SVL',
 'sex',
 'painted or not',
 'TOES',
 'RTL',
 'mass']

Now we will create an empty set, *uniquecols2*, that will eventually contain the unique column names in all of the files.

We will append the unique column names from each file to *uniquecols2*.

In [22]:
tmp = pd.Series(rawfiles).apply(xluniquecol2,verbose=False)
uniquecols2 = list()
for u in tmp:
    uniquecols2 = uniquecols2+u
uniquecols2 = list(set(uniquecols2))
uniquecols2

Check columns for file CC 2000-03-modified from CC-SJ 00-03 final.xls.


['TL',
 1,
 'species',
 'misc/notes',
 'VIAL',
 'paint mark',
 ' painted or not',
 'Year',
 'Toe 5',
 'Sex',
 'Mass',
 'Toe 12',
 'Misc.',
 'Location',
 'Toe 13',
 'Toes',
 '2015 or earlier',
 'Vial',
 'Date',
 'Collectors',
 'Marked',
 'Toe 10',
 'TIME',
 'Painted',
 'Toe 14',
 'TL (mm)',
 'NEW/recap',
 'misc',
 'RTL (mm)',
 'Toe 11',
 'Toe 15',
 '1st Capture (year)',
 'painted or not',
 'Toe 19',
 'mass',
 'New/Recap',
 'Toe 8',
 'Paint Mark',
 'Years Alive (known)',
 'Toe 6',
 'Unnamed: 17',
 'Meters',
 'Spotted',
 'painted',
 'Toe 20',
 'Toe 4',
 'Toe 3',
 'meters',
 'SVL',
 'Toe 16',
 'Time',
 'Toe 7',
 'TOES',
 'RTL',
 'sex',
 'Toe 17',
 'Toe 1',
 'Unnamed: 16',
 'Painted or Not',
 'Toe 2',
 'Mark',
 'SVL (mm)',
 'Unnamed: 19',
 'Toe 9',
 'mass (g)',
 'Toe 18',
 'Species',
 'location',
 'Tail condition (1=intact; 2=autotomized; 3=regrown)',
 'date']

<a id='DropCol'></a>

## Eliminate Unnecessary Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Now we will try to identify unnecessary columns and eliminate them. Much of this will be done manually.

In [23]:
keepCol = ['species', 'date', 'sex', 'svl', 'tl', 'rtl', 'mass',
       'paint.mark', 'location', 'meters', 'new.recap', 'painted', 'misc',
       'vial', 'autotomized', 'sighting', 'toes','filename']

In [24]:
set(pd.Series(keepCol).str.lower())-set(pd.Series(uniquecols2).str.lower())

{'autotomized', 'filename', 'new.recap', 'paint.mark', 'sighting'}

In [25]:
set(pd.Series(uniquecols2).str.lower())-set(pd.Series(keepCol).str.lower())

{' painted or not',
 '1st capture (year)',
 '2015 or earlier',
 'collectors',
 'mark',
 'marked',
 'mass (g)',
 'misc.',
 'misc/notes',
 nan,
 'new/recap',
 'paint mark',
 'painted or not',
 'rtl (mm)',
 'spotted',
 'svl (mm)',
 'tail condition (1=intact; 2=autotomized; 3=regrown)',
 'time',
 'tl (mm)',
 'toe 1',
 'toe 10',
 'toe 11',
 'toe 12',
 'toe 13',
 'toe 14',
 'toe 15',
 'toe 16',
 'toe 17',
 'toe 18',
 'toe 19',
 'toe 2',
 'toe 20',
 'toe 3',
 'toe 4',
 'toe 5',
 'toe 6',
 'toe 7',
 'toe 8',
 'toe 9',
 'unnamed: 16',
 'unnamed: 17',
 'unnamed: 19',
 'year',
 'years alive (known)'}

Since data for years 2000-2003 are contained in the same Excel file we will have to treat this file differently than the others.

<a id='CombineCol'></a>

## Combining Synonymous Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Once we have identified the columns we need to keep, we'll need to apply this list to the files as they are read into python by doing the following:

We will use a function, *colmatchtodict*,  to identify potential synonyms. Here's an example of how *colmatchtodict* works.  The code for this function can be found in [Functions](#functions).

In [26]:
coldict = {}

In [27]:
colmatchtodict('toes',pd.Series(uniquecols2),coldict, key = 'toes')

{'toes': ['Toes', 'TOES']}

Now let's see what happened when we apply this funtion to our, keepCol.

In [28]:
coldict = {}

In [29]:
pd.Series(keepCol).apply(lambda x: colmatchtodict(x=x,series=pd.Series(uniquecols2),dictsource=coldict))
coldict

{'species': ['species', 'Species'],
 'date': ['Date', 'date'],
 'sex': ['Sex', 'sex'],
 'svl': ['SVL', 'SVL (mm)'],
 'tl': ['TL', 'TL (mm)', 'RTL (mm)', 'RTL'],
 'rtl': ['RTL (mm)', 'RTL'],
 'mass': ['Mass', 'mass', 'mass (g)'],
 'paint.mark': ['paint mark', 'Paint Mark'],
 'location': ['Location', 'location'],
 'meters': ['Meters', 'meters'],
 'new.recap': ['NEW/recap', 'New/Recap'],
 'painted': [' painted or not',
  'Painted',
  'painted or not',
  'painted',
  'Painted or Not'],
 'misc': ['misc/notes', 'Misc.', 'misc'],
 'vial': ['VIAL', 'Vial'],
 'autotomized': ['Tail condition (1=intact; 2=autotomized; 3=regrown)'],
 'sighting': [],
 'toes': ['Toes', 'TOES'],
 'filename': []}

We will manually adjust the values for 'tl' and 'filename'.

In [30]:
coldict['tl']=['TL (mm)', 'TL', 'tl']

Now we need to use this dict to relabel the columns we wish to keep.

We will use the function, *findsyn* to identify potential synonymous to the columnlabels in *keepcols* among the column labels in *uniquecols2*. 

Here is are a few examples of how *findsyn* works.  The code can be found in [Functions](#functions).

In [31]:
findsyn('RTi',coldict,verbose=False)

In [32]:
findsyn('RTi',coldict,verbose=True)

No value matching "RTi" was found in the dictionary.


In [33]:
findsyn('RTL',coldict,verbose=True)

'rtl'

Now we apply *findsyn* to *uniquecol* and create a column of synonyms.

In [34]:
uniquecols2df = pd.DataFrame({'uniquecols2':uniquecols2})
uniquecols2df['preferredcol'] = uniquecols2df.uniquecols2.apply(lambda x: findsyn(x,coldict,False))
uniquecols2df

,uniquecols2,preferredcol
0,TL,tl
1,1,None
2,species,species
3,misc/notes,misc
4,VIAL,vial
5,paint mark,paint.mark
6,painted or not,painted
7,Year,None
8,Toe 5,None
9,Sex,sex


Now we will turn this dataframe back into a dict so that we can easily use it to rename columns

In [35]:
uniquecols2df.index = uniquecols2df.uniquecols2
uniquecols2dict = pd.Series(uniquecols2df.preferredcol).to_dict()
uniquecols2dict

{'TL': 'tl',
 1: None,
 'species': 'species',
 'misc/notes': 'misc',
 'VIAL': 'vial',
 'paint mark': 'paint.mark',
 ' painted or not': 'painted',
 'Year': None,
 'Toe 5': None,
 'Sex': 'sex',
 'Mass': 'mass',
 'Toe 12': None,
 'Misc.': 'misc',
 'Location': 'location',
 'Toe 13': None,
 'Toes': 'toes',
 '2015 or earlier': None,
 'Vial': 'vial',
 'Date': 'date',
 'Collectors': None,
 'Marked': None,
 'Toe 10': None,
 'TIME': None,
 'Painted': 'painted',
 'Toe 14': None,
 'TL (mm)': 'tl',
 'NEW/recap': 'new.recap',
 'misc': 'misc',
 'RTL (mm)': 'rtl',
 'Toe 11': None,
 'Toe 15': None,
 '1st Capture (year)': None,
 'painted or not': 'painted',
 'Toe 19': None,
 'mass': 'mass',
 'New/Recap': 'new.recap',
 'Toe 8': None,
 'Paint Mark': 'paint.mark',
 'Years Alive (known)': None,
 'Toe 6': None,
 'Unnamed: 17': None,
 'Meters': 'meters',
 'Spotted': None,
 'painted': 'painted',
 'Toe 20': None,
 'Toe 4': None,
 'Toe 3': None,
 'meters': 'meters',
 'SVL': 'svl',
 'Toe 16': None,
 'Time': None,

We'll use the dict, *uniquecols2dict* to rename the synonymous columns in our file....once we read them in,
that is.

<a id='ReadingAppendingData'></a>

# Reading and Appending Data
[Top](#TOC)

Now we use the function *readnsplit* to actually read in the source files, drop unnecessary columns and renaming columns according to a dictionary. 

Here is an example of how *readnsplit* works.  The code can be found in [Functions](#functions).

In [36]:
readnsplit(rawfiles[0],sourceinterDataBig,str)

Succes!  'CC 2000-03-modified from CC-SJ 00-03 final.xls',sheet '2000' has been saved to S:/Chris/TailDemography/TailDemography/Intermediate Files/Source and the corresponding                google drive file as S:/Chris/TailDemography/TailDemography/Intermediate Files/Source/CC 2000-03-modified from CC-SJ 00-03 final_2000.xls.
Succes!  'CC 2000-03-modified from CC-SJ 00-03 final.xls',sheet '2001' has been saved to S:/Chris/TailDemography/TailDemography/Intermediate Files/Source and the corresponding                google drive file as S:/Chris/TailDemography/TailDemography/Intermediate Files/Source/CC 2000-03-modified from CC-SJ 00-03 final_2001.xls.
Succes!  'CC 2000-03-modified from CC-SJ 00-03 final.xls',sheet '2002' has been saved to S:/Chris/TailDemography/TailDemography/Intermediate Files/Source and the corresponding                google drive file as S:/Chris/TailDemography/TailDemography/Intermediate Files/Source/CC 2000-03-modified from CC-SJ 00-03 final_2002.xls.
Succes!  '

In [37]:
for file in rawfiles:
    readnsplit(file,sourceinterDataBig,dtype=str, verbose=False)

We need to change the directory to the location where the intermediate files this operates on can be found.  We will also save a list of the files names in that location for convenience.

In [38]:
os.chdir(sourceinterDataBig)
splitfiles = glob.glob('*xls*')
splitfiles

['CC 2000-03-modified from CC-SJ 00-03 final_2000.xls',
 'CC 2000-03-modified from CC-SJ 00-03 final_2001.xls',
 'CC 2000-03-modified from CC-SJ 00-03 final_2002.xls',
 'CC 2000-03-modified from CC-SJ 00-03 final_2003.xls',
 'CC 2004_2004 .xlsx',
 'CC 2015 - captures_2015.xls',
 'CC 2016 - captures_2016.xls',
 'CC 2017 Lizards - 3viii17 captures and obs_2017.xls',
 'xCC2005x_2005.xls',
 'xCC2006x_2006.xls',
 'xCC2007x_2007.xls',
 'xCC2007x_Sheet1.xls',
 'xCC2008x_2008.xls',
 'xCC2009x_2009.xls',
 'xCC2010x_Sheet1.xlsx',
 'xCC2011x_Sheet1.xls',
 'xCC2012x_data.xls',
 'xCC2013x_CC 2013 data.xls',
 'xCC2014x_2014.xlsx']

Now we use the function *mapndrop* to drop unnecessary columns and renaming columns according to a dictionary.

Here are a few examples of how *mapndrop* works.  The code can be found in [Functions](#functions).

In [39]:
mapndrop(df=pd.read_excel(splitfiles[0],dtype=str),dictionary=uniquecols2dict,verbose = True)

Skipping mapndrop call for df.


In [40]:
mapndrop(df=pd.read_excel(splitfiles[4],dtype=str),dictionary=uniquecols2dict, verbose=True)

Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.


,species,toes,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial
0,sj,4-6-12-18-20,2004-07-02 00:00:00,m,88,89,-1,23.5,w..c,5m v 1 falls,-5,recap,painted,"B shed recently, T 1/2 shed(post not), loose s...",nan
1,sj,3-6-13-17,2004-07-02 00:00:00,f,66,99,0,9,w..a,L sb 7m v falls,-7,new,painted,"B shed recently, T soon",04 01
2,sj,3-6-13-20,2004-07-02 00:00:00,f,63,85,0,8.5,w..b,falls,0,new,painted,"B soon to shed, tail not",04 02
3,sv,3 13 16,2004-07-02 00:00:00,m,54,51,38,5.2,wTb,sb 10m v falls,-10,recap,painted,not shed since- paint still visible,nan
4,sj,2-7-11-17,2004-07-02 00:00:00,m,73,102,0,14,wAa,R outcrop bottom on rt cave trail,45,recap,painted,nan,nan
5,sv,4 6 11,2004-07-02 00:00:00,m,49,66,0,4.5,wAc,see wAa,45,new,painted,nan,04 03
6,sv,4 6 12,2004-07-02 00:00:00,m,45,62,0,3.3,wAb,3 m above cave trail on rt,48,new,painted,nan,04 04
7,sj,3-6-14-17,2004-07-02 00:00:00,f,60,86,0,7.5,wA1a,bottom wall v old pine xing,nan,new,painted,B shed; T soon to shed,04 05
8,sj,3-6-15-17,2004-07-02 00:00:00,f,59,77,0,6.6,wBc,top wall right of pine xing,nan,new,painted,nan,04 06
9,sj,2-8-11-19,2004-07-02 00:00:00,m,70,101,0,12.3,wBb,middle wall v pine xing 1m^ fox hole,nan,recap,painted,may have recently shed recently,nan


We'll create a df, *df*, with no data, but columns from our desired columns, *i.e.* the keys for coldict, as a placeholder to which we can append new data.

In [41]:
df = pd.DataFrame(columns=coldict.keys())
df

,species,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial,autotomized,sighting,toes,filename


Now we will read in all of the successfully split files, clean the column names, and concatenate them into one large df.

In [42]:
for file in splitfiles:
    df = pd.concat([df,mapndrop(pd.read_excel(file,dtype=str),uniquecols2dict)],sort=True)
    print(df.shape[0])
print("\n\nFinal df has {} columns and {} rows.".format(df.shape[1],df.shape[0]))
df.head()

Skipping mapndrop call for df.
0
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
681
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
2158
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3175
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3654
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3895
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
3998
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
4796
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
4998
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
5161
Successfully mapped columns for df.
Successfully dropped unnecessary columns for df.
5343
Successfully mapped columns for df.
Successfully dropped unnecessary

,autotomized,date,filename,location,mass,meters,misc,new.recap,paint.mark,painted,rtl,sex,sighting,species,svl,tl,toes,vial
0,NaN,2001-03-18 00:00:00,NaN,r wall at juniper xing,13.7,113,nan,NEW,w.c,nan,1,m,NaN,sj,76,80,2-6-11-16,toes in vial 01-1
1,NaN,2001-03-18 00:00:00,NaN,r wall at juniper xing,17.5,113,nan,NEW,w.b,nan,0,f,NaN,sj,82,109,2-6-11-18,toes in vial 01-2
2,NaN,2001-03-18 00:00:00,NaN,r wall at juniper xing,8.5,113,nan,NEW,w.a,nan,-1,f,NaN,sj,58,69,2-6-11-19,toes in vial 01-3
3,NaN,2001-03-18 00:00:00,NaN,r wall at juniper xing,9.2,113,nan,NEW,w-a,nan,0,m,NaN,sj,65,91,2-6-11-20,toes in vial 01-4
4,NaN,2001-03-18 00:00:00,NaN,r wall at juniper xing,7.8,113,nan,NEW,w-b,nan,0,f,NaN,sj,58,76,2-6-11-19-20,toes in vial 01-5


In [43]:
df = df.reindex(['species', 'toes', 'sex', 'date', 'svl', 'tl', 'rtl', 'autotomized', 'mass', 
                 'location', 'meters', 'new.recap', 'painted', 'sighting', 
                 'paint.mark', 'vial', 'misc'], axis=1)
df.head()

,species,toes,sex,date,svl,tl,rtl,autotomized,mass,location,meters,new.recap,painted,sighting,paint.mark,vial,misc
0,sj,2-6-11-16,m,2001-03-18 00:00:00,76,80,1,NaN,13.7,r wall at juniper xing,113,NEW,nan,NaN,w.c,toes in vial 01-1,nan
1,sj,2-6-11-18,f,2001-03-18 00:00:00,82,109,0,NaN,17.5,r wall at juniper xing,113,NEW,nan,NaN,w.b,toes in vial 01-2,nan
2,sj,2-6-11-19,f,2001-03-18 00:00:00,58,69,-1,NaN,8.5,r wall at juniper xing,113,NEW,nan,NaN,w.a,toes in vial 01-3,nan
3,sj,2-6-11-20,m,2001-03-18 00:00:00,65,91,0,NaN,9.2,r wall at juniper xing,113,NEW,nan,NaN,w-a,toes in vial 01-4,nan
4,sj,2-6-11-19-20,f,2001-03-18 00:00:00,58,76,0,NaN,7.8,r wall at juniper xing,113,NEW,nan,NaN,w-b,toes in vial 01-5,nan


In [44]:
df.shape

(6597, 17)

<a id='ExportingData'></a>

# Exporting Data
[Top](#TOC)

Here we call the function, *namefile*, to create a timestamped name for file to be exported.  You can find the code for *namefile* in [Functions](#functions).

In [45]:
filename = namefile('Appended and Trimmed CC Data 2000-2017')
os.chdir(outputBig)
df.to_csv(filename,index = False)
print("\'{}\' has been saved to \'{}\' and the corresponding drive google drive location."\
      .format(filename, outputBig))

'Appended and Trimmed CC Data 2000-2017_2019-01-01 14hrs11min.csv' has been saved to 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data' and the corresponding drive google drive location.
